# 스태킹 - "위스콘신 암 데이터 셋"

## - "개별 모델" : KNN, 랜덤 포레스트, 결정 트리, 에이다 부스트
## - "최종 모델" : 로지스틱 회귀

## 1. 기본 스태킹 모델

## (1) 데이터 불러오기

In [8]:
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

cancer_data = load_breast_cancer()

X_data = cancer_data.data
y_label = cancer_data.target

X_train, X_test, y_train, y_test = train_test_split(X_data, y_label, test_size=0.2, random_state=0)

## (2) 개별 모델들로 데이터 셋을 학습 및 예측

In [9]:
#개별 모델
knn_clf = KNeighborsClassifier(n_neighbors=4)
rf_clf = RandomForestClassifier(n_estimators=100, random_state=0)
dt_clf = DecisionTreeClassifier()
ada_clf = AdaBoostClassifier(n_estimators=100)

#최종 모델
lr_final = LogisticRegression(C=10)

#개별 모델들 학습 및 예측
knn_clf.fit(X_train, y_train)
rf_clf.fit(X_train, y_train)
dt_clf.fit(X_train, y_train)
ada_clf.fit(X_train, y_train)

knn_pred = knn_clf.predict(X_test)
rf_pred = rf_clf.predict(X_test)
dt_pred = dt_clf.predict(X_test)
ada_pred = ada_clf.predict(X_test)

print('KNN 정확도 : {0:.4f}'.format(accuracy_score(y_test, knn_pred)))
print('랜덤 포레스트 정확도 : {0:.4f}'.format(accuracy_score(y_test, rf_pred)))
print('결정 트리 정확도 : {0:.4f}'.format(accuracy_score(y_test, dt_pred)))
print('에이다부스트 정확도 : {0:.4f}'.format(accuracy_score(y_test, ada_pred)))

KNN 정확도 : 0.9211
랜덤 포레스트 정확도 : 0.9649
결정 트리 정확도 : 0.9035
에이다부스트 정확도 : 0.9561


## (3) 개별 모델들의 예측결과 컬럼들을 모두 옆으로 붙여 새로운 메타데이터를 만든다.

In [10]:
pred = np.array([knn_pred, rf_pred, dt_pred, ada_pred])
print(pred.shape)

(4, 114)


In [11]:
#transpose를 이용해 칼럼 별로 각 개별 모델의 예측값이 들어가도록 한다.
pred = np.transpose(pred)
print(pred.shape)

(114, 4)


## (4) 최종 모델로 3.에서 만든 메타데이터 학습 후 테스트 데이터 예측하기.

In [29]:
pred_train, pred_test, y_test_train, y_test_test = train_test_split(pred, y_test, test_size=0.2, random_state=0)

lr_final.fit(pred_train, y_test_train)
final = lr_final.predict(pred_test)

print('최종 메타 모델의 예측 정확도 : {0:.4f}'.format(accuracy_score(y_test_test, final)))

최종 메타 모델의 예측 정확도 : 0.8696


random_state를 다른 값으로 하면 정확도가 올라가는데 0일때만 값이 낮다.

## 2. CV 셋 기반의 스태킹

방법이 조금 헷갈릴 수 있는데, 이해한 것을 그대로 써볼게. <br>
우선 개별 모델이 여러개 있다고 가정하자. <br>
이때 각각의 개별 모델들에게 다음 과정이 수행된다. <br>
<p></p>
우선 "데이터 셋 = 학습 셋 + 테스트 셋"으로 나누어진다. <br>
<p></p>
(step 1) 이때 Folds의 개수가 n이라면, 학습셋을 n개의 폴드로 나누고 그 중 한 폴드씩 검증 폴드로 이용하고 나머지 폴드는 학습 포드로 이용하여 학습 후 검증 폴드로 예측한 결과값을 모두 저장하면, 학습셋의 row의 개수와 동일한 예측결과값이 저장된다. <br> 
<p></p>
(step 2) 그리고 각 학습폴드별 학습 수행 시 테스트 셋을 이용해서도 예측한 결과를 또 저장하고 모두 마치면 테스트 셋을 이용한 예측결과의 평균을 저장한다. 그러면 또한 테스트 셋의 row개수와 동일한 예측결과값이 나온다.<br>
<p></p>
각각의 개별 모델로 부터 위의 예측결과 값을 모두 모아, <br> 
step 1에서 나온 결과를 모두 옆으로 쌓아 최종모델 학습용 메타 데이터를 만든다. <br>
step 2에서 나온 결과를 모두 옆으로 쌓아 최종모델 테스트용 메타 데이터를 만든다. <br>
<p></p>
해당 메타 데이터로 최종모델을 이용해 학습 / 예측을 진행한다.


## (1) 각 개별 모델 별 메타데이터 만들기

In [35]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

#개별 모델의 최종 메타 모델이 사용할 학습 및 테스트용 데이터를 만드는 함수
def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_folds):
    #지정된 폴드값으로 폴드생성
    kf = KFold(n_splits=n_folds, shuffle=False)
    
    #추후의 최종 메타 모델이 사용할 학습 데이터 반환을 위한 넘파이 배열 초기화
    train_fold_pred = np.zeros((X_train_n.shape[0], 1))
    test_pred = np.zeros((X_test_n.shape[0], n_folds)) # 이후 이 배열의 평균을 구해 columns이 1짜리로 다시 만들어 리턴할 것임
    print(model.__class__.__name__, ' model 시작 ')
    
    #각 폴드마다 돌아가면서 학습폴드로 학습하고, 검증폴더로 예측한 결과를 train_fold_pred에 각 index에 찾아서 저장(KFold는 분리한 데이터를 index로 반환하기 때문)하고,
    #학습된 모델을 테스트 셋으로 예측한 뒤 test_pred에 각 행(폴드번호)에 저장한다.
    for folder_counter, (train_index, valid_index) in enumerate(kf.split(X_train_n)): 
        print('\t 폴드 세트 : ', folder_counter, ' 시작 ')
        X_tr = X_train_n[train_index]
        y_tr = y_train_n[train_index]
        X_te = X_train_n[valid_index]
        
        model.fit(X_tr, y_tr)
        train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1, 1)
        test_pred[:, folder_counter] = model.predict(X_test_n)
        
    #학습된 모델을 테스트 셋으로 예측한 뒤 test_pred에 각 행(폴드번호)에 저장한 것을 행끼리 평균낸다.
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1, 1)
    
    return train_fold_pred, test_pred_mean

In [36]:
knn_train, knn_test = get_stacking_base_datasets(knn_clf, X_train, y_train, X_test, 7)
rf_train, rf_test = get_stacking_base_datasets(rf_clf, X_train, y_train, X_test, 7)
dt_train, dt_test = get_stacking_base_datasets(dt_clf, X_train, y_train, X_test, 7)
ada_train, ada_test = get_stacking_base_datasets(ada_clf, X_train, y_train, X_test, 7)

KNeighborsClassifier  model 시작 
	 폴드 세트 :  0  시작 
	 폴드 세트 :  1  시작 
	 폴드 세트 :  2  시작 
	 폴드 세트 :  3  시작 
	 폴드 세트 :  4  시작 
	 폴드 세트 :  5  시작 
	 폴드 세트 :  6  시작 
RandomForestClassifier  model 시작 
	 폴드 세트 :  0  시작 
	 폴드 세트 :  1  시작 
	 폴드 세트 :  2  시작 
	 폴드 세트 :  3  시작 
	 폴드 세트 :  4  시작 
	 폴드 세트 :  5  시작 
	 폴드 세트 :  6  시작 
DecisionTreeClassifier  model 시작 
	 폴드 세트 :  0  시작 
	 폴드 세트 :  1  시작 
	 폴드 세트 :  2  시작 
	 폴드 세트 :  3  시작 
	 폴드 세트 :  4  시작 
	 폴드 세트 :  5  시작 
	 폴드 세트 :  6  시작 
AdaBoostClassifier  model 시작 
	 폴드 세트 :  0  시작 
	 폴드 세트 :  1  시작 
	 폴드 세트 :  2  시작 
	 폴드 세트 :  3  시작 
	 폴드 세트 :  4  시작 
	 폴드 세트 :  5  시작 
	 폴드 세트 :  6  시작 


## (2) 각 개별 모델별로 만든 메타데이터 합치기

In [39]:
Stack_final_X_train = np.concatenate((knn_train, rf_train, dt_train, ada_train), axis=1)
Stack_final_X_test = np.concatenate((knn_test, rf_test, dt_test, ada_test), axis=1)
print('원본 학습 피처 데이터 shape : ', X_train.shape, '원본 테스트 피처 shape : ', X_test.shape)
print('스태킹 학습 피처 데이터 shape : ', Stack_final_X_train.shape, '스태킹 테스트 피처 데이터 shape : ', Stack_final_X_test.shape)

원본 학습 피처 데이터 shape :  (455, 30) 원본 테스트 피처 shape :  (114, 30)
스태킹 학습 피처 데이터 shape :  (455, 4) 스태킹 테스트 피처 데이터 shape :  (114, 4)


각각의 개별 모델의 에측결과를 합쳤으므로 메타데이터의 column개수는 개별모델의 개수인 4개이다.

## (3) 최종 모델로 메타데이터 학습 / 예측하기

In [40]:
lr_final.fit(Stack_final_X_train, y_train)
stack_final = lr_final.predict(Stack_final_X_test)

print('최종 메타 모델의 예측 정확도 : {0:.4f}'.format(accuracy_score(y_test, stack_final)))

최종 메타 모델의 예측 정확도 : 0.9737


# 파라미터 튜닝?

여기서는 파라미터 튜닝을 실시하지 않았지만, 일반적으로 파라미터 튜닝을 최적으로 해놓고 스태킹 앙상블 기법을 이용한다. 이때 파라미터를 튜닝한다는 것은 개별 모델의 파라미터를 최적으로 튜닝하는 것을 의미한다.